In [12]:
from time import sleep
import google.generativeai as genai
import configparser
import json
import re
import pandas as pd

In [2]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
consulta = """
"Buenas tardes, 
al intentar iniciar el tramite de renovación de habilitación Municipal sin modificaciones el sistema nos devuelve el siguiente mensaje:
""Advertencia
Deberás realizar el Trámite de Renovación con modificaciones - Modificar Actividad porque hemos detectado diferencias entre la/s actividad/es y el/los destino/s del último certificado otorgado.""
como debemos proceder para renovar la habilitación y cuales son las diferencias detectadas?
Gracias
Saludos
"""

prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
    )

print(response.text)

Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [ ]:
data = pd.read_excel('/home/gcasado0/proyectos/chatbot/Consultas_habilitaciones.xlsx')
data.head()

In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    # ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)
    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
    consulta = data['Contenido consulta'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue    
    prompt = f"""
    Extraer un resumen de lo que quiere el remitente en la siguiente consulta, que está delimitada por tres comillas invertidas.
    Extraer los temas principales de la consulta.
    Haga que cada elemento tenga una o dos palabras. Formatee su respuesta como una lista de elementos separados por comas.
    Indique el sentimiento general del texto.
    Indique si el remitente está expresando enojo.
    Indique si la consulta es simple de resolver o no.
    Indique si el remitente no sabe que actividad elegir para su habilitacion.

    Texto: '''{consulta}'''
    Salida en formato JSON:
    "Resumen":
    "Temas": 
    "Sentimientos": 
    "Enojo": "Si/No"
    "Complejidad": "Simple/Compleja"
    "No sabe actividad": "Si/No"
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Resumen'] = analisis.get('Resumen', 'None')
            data.loc[i, 'Temas'] = analisis.get('Temas', [])
            data.loc[i, 'Complejidad'] = analisis.get('Complejidad', 'None')
            data.loc[i, 'Sentimientos'] = analisis.get('Sentimientos', 'None')
            data.loc[i, 'Enojo'] = analisis.get('Enojo', 'None')
            # No sabe actividad
            data.loc[i, 'No sabe actividad'] = analisis.get('No sabe actividad', 'None')

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")

# extraer del dataframe solo las siguientes columnas: 'Contenido consulta', 'Resumen', 'Temas', 'Sentimientos', 'Enojo'
data_analizado = data[['Temática ','Contenido consulta', 'Resumen', 'Temas','Complejidad', 'Sentimientos', 'Enojo','No sabe actividad']]

# guardar el dataframe data con las respuestas en un archivo Excel
data_analizado.to_excel('Consultas_habilitaciones_resumen.xlsx', index=False)
